# Fine-tune BART for Summarization

We will use the CNN/DailyMail dataset and fine-tune the `facebook/bart-base` model using Hugging Face.

In [ ]:
# Install required packages (if not already installed)
!pip install datasets transformers rouge_score accelerate -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
dataset = load_dataset("xsum")
train_dataset = dataset["train"]
val_dataset = dataset["validation"]

In [ ]:
# Load tokenizer and model
model_checkpoint = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
# Preprocessing
max_input_length = 512
max_target_length = 128

def preprocess(examples):
    inputs = tokenizer(examples["document"], max_length=max_input_length, truncation=True)
    targets = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True)
    inputs["labels"] = targets["input_ids"]
    return inputs

In [ ]:
small_train = train_dataset.select(range(1000))
small_val = val_dataset.select(range(200))
train_dataset=small_train
val_dataset=small_val

In [ ]:
# Tokenize the datasets
tokenized_train = train_dataset.map(preprocess, batched=True, remove_columns=train_dataset.column_names)
tokenized_val = val_dataset.map(preprocess, batched=True, remove_columns=val_dataset.column_names)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
!pip install evaluate -q

In [ ]:
# Define ROUGE metric
import evaluate
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return rouge.compute(predictions=decoded_preds, references=decoded_labels)

In [ ]:
# Training arguments
args = TrainingArguments(
    output_dir="./results",
    # evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    # predict_with_generate=True,
    fp16=True
)

In [ ]:
!pip install -U transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 65.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.3
    Uninstalling transformers-4.50.3:
      Successfully uninstalled transformers-4.50.3


In [ ]:
# Trainer
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # predict_with_generate=True
)

<ipython-input-26-16a03c084782>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Start training
trainer.train()

Step,Training Loss


TrainOutput(global_step=250, training_loss=2.190537109375, metrics={'train_runtime': 58.6042, 'train_samples_per_second': 17.064, 'train_steps_per_second': 4.266, 'total_flos': 291577879756800.0, 'train_loss': 2.190537109375, 'epoch': 1.0})

In [ ]:
# Save model
model.save_pretrained("./finetuned-bart-cnn")
tokenizer.save_pretrained("./finetuned-bart-cnn")

('./finetuned-bart-cnn/tokenizer_config.json',
 './finetuned-bart-cnn/special_tokens_map.json',
 './finetuned-bart-cnn/vocab.json',
 './finetuned-bart-cnn/merges.txt',
 './finetuned-bart-cnn/added_tokens.json',
 './finetuned-bart-cnn/tokenizer.json')

In [ ]:
import random

In [ ]:
print("\n\n==== Testing on Random Article ====\n")

sample = random.choice(val_dataset)
print("\n\033[1mOriginal Article:\033[0m\n", sample['document'])
print("\n\033[1mReference Summary:\033[0m\n", sample['summary'])




==== Testing on Random Article ====


Original Article:
 The hyperbaric chamber, which treats divers with "the bends", was operated by St John's Ambulance on a donation basis until it broke in April 2014.
The health department replaced it in 2015, but says it needs to "balance the books".
Diving instructor Steve Bougourd said he was "gobsmacked".
"I'm just worried that this kind of cost will put people off of actually going to the [hospital] and notifying them if they suspect a problem," he said.
"We may find it's going to be very expensive to get out divers insured."
In the UK hyperbaric oxygen treatment is covered by the NHS, but Guernsey has its own health care system.
Source: NHS
Assistant director at Guernsey's health and social care department (HSC) Ed Freestone said renting the chamber was costing the government £60,000 a year.
He said the department would not make a profit from the new charges, which were based on "the average usage that we could identify over the previous fe

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Tokenize and move input to same device
inputs = tokenizer(sample['document'], return_tensors="pt", max_length=512, truncation=True)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Generate summary
summary_ids = model.generate(inputs["input_ids"], max_length=64, num_beams=4, early_stopping=True)
predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("\n\033[1mPredicted Summary:\033[0m\n", predicted_summary)


Predicted Summary:
 The health department of Guernsey is planning to buy its own hyperbaric chamber to pay £60,000 a year.
